In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    !pip install unsloth
    !pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
    !pip install pandas numpy
    !pip install causal-learn dowhy
    !pip install torch transformers bitsandbytes
    !pip install datasets

In [2]:
import unsloth
import torch
from unsloth import FastLanguageModel
from trl import SFTTrainer
from unsloth import is_bfloat16_supported
import pandas as pd
import random
import os
import numpy as np
from datasets import Dataset
from transformers import TrainingArguments
import re
import random

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
# Parse the causal relationships from the causal_effects.csv file
def parse_causal_relationships(causal_effects_df):
    causal_relations = []

    for _, row in causal_effects_df.iterrows():
        treatment = row['treatment']
        outcome = row['outcome']
        effect = row['effect']
        stderr = row['stderr'] if not pd.isna(row['stderr']) else None
        p_value = row['p_value'] if not pd.isna(row['p_value']) else None

        # Determine relationship type based on effect magnitude and sign
        if abs(effect) < 0.001:  # Very small effect, consider as minimal causal relationship
            relation_type = "≈0"
        elif effect > 0:
            relation_type = "+"
        else:
            relation_type = "-"

        # Store the magnitude for interpretation
        magnitude = abs(effect)

        causal_relations.append({
            'treatment': treatment,
            'outcome': outcome,
            'effect': effect,
            'relation_type': relation_type,
            'magnitude': magnitude,
            'stderr': stderr,
            'p_value': p_value
        })

    return causal_relations

# Get the list of excluded variables (those not found in any causal relationship)
def get_excluded_variables(causal_relations):
    all_variables = set()

    for rel in causal_relations:
        all_variables.add(rel['treatment'])
        all_variables.add(rel['outcome'])

    # You might want to exclude variables with very few connections or with special properties
    # For now, we're not excluding any variables found in the dataset
    excluded_variables = []

    return excluded_variables

# Generate training examples from causal relationships
def generate_training_examples(causal_relations, excluded_variables, num_examples=5000):
    examples = []

    # Define interpretation frameworks for effects
    def interpret_effect(effect, magnitude):
        # Sign interpretation
        if abs(effect) < 0.001:
            sign_desc = "has negligible or no causal effect on"
        elif effect > 0:
            sign_desc = "positively causes" if magnitude > 1 else "slightly increases"
        else:
            sign_desc = "negatively affects" if magnitude > 1 else "slightly decreases"

        # Magnitude interpretation
        if magnitude > 10:
            strength = "very strong"
        elif magnitude > 1:
            strength = "strong"
        elif magnitude > 0.1:
            strength = "moderate"
        elif magnitude > 0.01:
            strength = "weak"
        else:
            strength = "very weak"

        return sign_desc, strength

    # Variables that appear in the causal relations
    valid_variables = set()
    for rel in causal_relations:
        valid_variables.add(rel['treatment'])
        valid_variables.add(rel['outcome'])

    # Remove excluded variables from valid variables
    valid_variables = valid_variables - set(excluded_variables)
    valid_variables = list(valid_variables)

    # Create direct lookup dictionary for faster access
    causal_lookup = {}
    for rel in causal_relations:
        if rel['treatment'] in excluded_variables or rel['outcome'] in excluded_variables:
            continue

        key = (rel['treatment'], rel['outcome'])
        causal_lookup[key] = rel

    # Generate examples
    for _ in range(num_examples):
        # Choose example type
        example_type = random.choice([
            "direct_relation",
            "reverse_relation",
            "non_relation",
            "complex_query"
        ])

        if example_type == "direct_relation":
            # Pick a random relation from the dataset
            rel = random.choice(causal_relations)
            treatment, outcome = rel['treatment'], rel['outcome']
            effect = rel['effect']
            magnitude = rel['magnitude']

            if treatment in excluded_variables or outcome in excluded_variables:
                continue

            # Format variable names for readability
            treatment_readable = treatment.replace("_", " ")
            outcome_readable = outcome.replace("_", " ")

            sign_desc, strength = interpret_effect(effect, magnitude)

            question = f"Is there a causal relationship between '{treatment_readable}' and '{outcome_readable}'? If yes, describe the relationship."
            answer = f"Yes, there is a causal relationship. '{treatment_readable}' {sign_desc} '{outcome_readable}' with a {strength} effect (coefficient: {effect:.4f})."

        elif example_type == "reverse_relation":
            # Look for a relation and ask about it in reverse
            rel = random.choice(causal_relations)
            treatment, outcome = rel['treatment'], rel['outcome']

            if treatment in excluded_variables or outcome in excluded_variables:
                continue

            treatment_readable = treatment.replace("_", " ")
            outcome_readable = outcome.replace("_", " ")

            question = f"Is there a causal relationship between '{outcome_readable}' and '{treatment_readable}'? If yes, describe the relationship."

            # Check if there's a reverse relation
            reverse_key = (outcome, treatment)
            if reverse_key in causal_lookup:
                reverse_rel = causal_lookup[reverse_key]
                reverse_effect = reverse_rel['effect']
                reverse_magnitude = reverse_rel['magnitude']
                reverse_sign_desc, reverse_strength = interpret_effect(reverse_effect, reverse_magnitude)

                answer = f"Yes, there is a causal relationship. '{outcome_readable}' {reverse_sign_desc} '{treatment_readable}' with a {reverse_strength} effect (coefficient: {reverse_effect:.4f})."
            else:
                # Check for bidirectional relationships or provide clear negative
                effect = rel['effect']
                magnitude = rel['magnitude']
                sign_desc, strength = interpret_effect(effect, magnitude)

                answer = f"Based on the available data, there's no detected causal relationship from '{outcome_readable}' to '{treatment_readable}'. However, the data shows that '{treatment_readable}' {sign_desc} '{outcome_readable}' with a {strength} effect (coefficient: {effect:.4f})."

        elif example_type == "non_relation":
            # Pick two random variables that don't have a direct relation
            while True:
                treatment = random.choice(valid_variables)
                outcome = random.choice(valid_variables)

                if treatment != outcome and (treatment, outcome) not in causal_lookup and (outcome, treatment) not in causal_lookup:
                    break

            treatment_readable = treatment.replace("_", " ")
            outcome_readable = outcome.replace("_", " ")

            question = f"Is there a causal relationship between '{treatment_readable}' and '{outcome_readable}'? If yes, describe the relationship."
            answer = f"Based on the available data, there's no detected causal relationship between '{treatment_readable}' and '{outcome_readable}'."

        else:  # complex_query
            # Generate a more complex causal query
            query_type = random.choice([
                "effect_magnitude",
                "causal_path",
                "effect_comparison"
            ])

            if query_type == "effect_magnitude":
                # Find variables with strong effects
                strong_effects = [rel for rel in causal_relations if abs(rel['effect']) > 1.0]

                if strong_effects:
                    rel = random.choice(strong_effects)
                    treatment, outcome = rel['treatment'], rel['outcome']
                    effect = rel['effect']
                    magnitude = rel['magnitude']

                    if treatment in excluded_variables or outcome in excluded_variables:
                        # Fallback
                        rel = random.choice(causal_relations)
                        treatment, outcome = rel['treatment'], rel['outcome']
                        effect = rel['effect']
                        magnitude = rel['magnitude']

                    treatment_readable = treatment.replace("_", " ")
                    outcome_readable = outcome.replace("_", " ")
                    sign_desc, strength = interpret_effect(effect, magnitude)

                    question = f"How strong is the causal effect of '{treatment_readable}' on '{outcome_readable}'? Please interpret the magnitude."
                    answer = f"The causal effect of '{treatment_readable}' on '{outcome_readable}' is {strength}. With a coefficient of {effect:.4f}, an increase in '{treatment_readable}' {sign_desc.split(' ')[-2:]} '{outcome_readable}'. This indicates a {strength} causal relationship."
                else:
                    # Fallback
                    rel = random.choice(causal_relations)
                    treatment, outcome = rel['treatment'], rel['outcome']
                    effect = rel['effect']
                    magnitude = rel['magnitude']
                    treatment_readable = treatment.replace("_", " ")
                    outcome_readable = outcome.replace("_", " ")
                    sign_desc, strength = interpret_effect(effect, magnitude)

                    question = f"How strong is the causal effect of '{treatment_readable}' on '{outcome_readable}'? Please interpret the magnitude."
                    answer = f"The causal effect of '{treatment_readable}' on '{outcome_readable}' is {strength}. With a coefficient of {effect:.4f}, an increase in '{treatment_readable}' {sign_desc.split(' ')[-2:]} '{outcome_readable}'. This indicates a {strength} causal relationship."

            elif query_type == "causal_path":
                # Try to find a causal path (A -> B -> C)
                paths = []
                for rel1 in causal_relations:
                    treatment_a = rel1['treatment']
                    outcome_b = rel1['outcome']

                    for rel2 in causal_relations:
                        treatment_b = rel2['treatment']
                        outcome_c = rel2['outcome']

                        if outcome_b == treatment_b:
                            # Found a path A -> B -> C
                            if treatment_a not in excluded_variables and outcome_b not in excluded_variables and outcome_c not in excluded_variables:
                                paths.append((rel1, rel2))

                if paths:
                    path_pair = random.choice(paths)
                    rel1, rel2 = path_pair

                    treatment_a = rel1['treatment']
                    outcome_b = rel1['outcome']  # same as treatment_b
                    outcome_c = rel2['outcome']

                    effect1 = rel1['effect']
                    effect2 = rel2['effect']
                    combined_effect = effect1 * effect2  # simplified estimate of combined effect

                    a_readable = treatment_a.replace("_", " ")
                    b_readable = outcome_b.replace("_", " ")
                    c_readable = outcome_c.replace("_", " ")

                    sign_desc1, strength1 = interpret_effect(effect1, abs(effect1))
                    sign_desc2, strength2 = interpret_effect(effect2, abs(effect2))
                    sign_desc_combined, strength_combined = interpret_effect(combined_effect, abs(combined_effect))

                    question = f"Is there an indirect causal relationship from '{a_readable}' to '{c_readable}'? Explain the causal pathway if any."
                    answer = f"Yes, there is an indirect causal relationship from '{a_readable}' to '{c_readable}'. The causal pathway is: '{a_readable}' {sign_desc1} '{b_readable}' (coefficient: {effect1:.4f}), which in turn {sign_desc2} '{c_readable}' (coefficient: {effect2:.4f}). Through this pathway, '{a_readable}' has an estimated indirect effect on '{c_readable}' that is {strength_combined}."
                else:
                    # Fallback
                    vars_sample = random.sample(valid_variables, 2)
                    a_readable = vars_sample[0].replace("_", " ")
                    c_readable = vars_sample[1].replace("_", " ")

                    question = f"Is there an indirect causal relationship from '{a_readable}' to '{c_readable}'? Explain the causal pathway if any."
                    answer = f"Based on the available data, there's no clear indirect causal pathway from '{a_readable}' to '{c_readable}'."

            else:  # effect_comparison
                # Compare the effects of a variable on multiple outcomes
                effects_map = {}
                for rel in causal_relations:
                    treatment = rel['treatment']
                    if treatment in excluded_variables:
                        continue

                    if treatment not in effects_map:
                        effects_map[treatment] = []

                    outcome = rel['outcome']
                    if outcome not in excluded_variables:
                        effects_map[treatment].append((outcome, rel['effect'], rel['magnitude']))

                treatments_with_multiple_effects = [treatment for treatment, effects in effects_map.items() if len(effects) >= 2]

                if treatments_with_multiple_effects:
                    treatment = random.choice(treatments_with_multiple_effects)
                    effects = random.sample(effects_map[treatment], 2)  # Take just 2 effects for simplicity

                    treatment_readable = treatment.replace("_", " ")
                    outcome1_readable = effects[0][0].replace("_", " ")
                    outcome2_readable = effects[1][0].replace("_", " ")
                    effect1 = effects[0][1]
                    effect2 = effects[1][1]
                    magnitude1 = effects[0][2]
                    magnitude2 = effects[1][2]

                    sign_desc1, strength1 = interpret_effect(effect1, magnitude1)
                    sign_desc2, strength2 = interpret_effect(effect2, magnitude2)

                    question = f"What are the causal effects of '{treatment_readable}' on other variables? Compare at least two effects."
                    answer = f"'{treatment_readable}' has different causal effects on multiple variables. It {sign_desc1} '{outcome1_readable}' with a {strength1} effect (coefficient: {effect1:.4f}) and {sign_desc2} '{outcome2_readable}' with a {strength2} effect (coefficient: {effect2:.4f}). These effects show that '{treatment_readable}' has {'stronger' if magnitude1 > magnitude2 else 'weaker'} influence on '{outcome1_readable}' compared to '{outcome2_readable}'."
                else:
                    # Fallback
                    treatment = random.choice(valid_variables)
                    treatment_readable = treatment.replace("_", " ")

                    question = f"What are the causal effects of '{treatment_readable}' on other variables? List any effects found."
                    answer = f"Based on the available data, there are limited or no clear causal effects of '{treatment_readable}' on other variables identified."

        examples.append({
            'question': question,
            'answer': answer
        })

    return examples

# Format training examples for instruction tuning
def format_for_instruction_tuning(examples):
    formatted_examples = []

    for ex in examples:
        formatted = {
            "text": f"<|system|>\nYou are a causal inference expert that answers questions about causal relationships between variables based on data analysis. When asked about relationships, provide clear explanations about whether one variable causes another, is caused by another, or shares common causes with other variables. Interpret both the sign (positive/negative) and magnitude (strong/weak) of causal effects.\n<|user|>\n{ex['question']}\n<|assistant|>\n{ex['answer']}"
        }
        formatted_examples.append(formatted)

    return formatted_examples

# Main fine-tuning setup
def main():
    # Load the causal effects data
    causal_effects_df = pd.read_csv("causal_effects.csv")

    # Parse data
    causal_relations = parse_causal_relationships(causal_effects_df)
    excluded_variables = get_excluded_variables(causal_relations)

    # Generate training examples
    training_examples = generate_training_examples(causal_relations, excluded_variables)

    # Format for instruction tuning
    formatted_examples = format_for_instruction_tuning(training_examples)

    # Convert to Hugging Face dataset
    dataset = Dataset.from_pandas(pd.DataFrame(formatted_examples))

    # Split into train and eval
    split_dataset = dataset.train_test_split(test_size=0.1)

    # Initialize Unsloth with Llama 3.1 8B
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="unsloth/Meta-Llama-3.1-8B",
        max_seq_length=2048,
        dtype=None,  # Set to torch.bfloat16 or None based on your GPU capabilities
        load_in_4bit=True,  # Use 4-bit quantization to reduce memory requirements
    )

    # Setup LoRA fine-tuning
    model = FastLanguageModel.get_peft_model(
        model,
        r=16,  # LoRA attention dimension
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                       "gate_proj", "up_proj", "down_proj"],
        lora_alpha=32,
        lora_dropout=0.05,
        bias="none",
    )

    trainer = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=split_dataset['train'],  # Use the train split explicitly
        eval_dataset=split_dataset['test'],    # Use the test split explicitly
        dataset_text_field="text",
        dataset_num_proc=2,
        packing=False,
        args=TrainingArguments(
            output_dir="./causal-inference-llama-3.1",
            per_device_train_batch_size=2,
            per_device_eval_batch_size=2,
            gradient_accumulation_steps=4,
            warmup_steps=5,
            num_train_epochs=3,
            max_steps=60,
            learning_rate=2e-4,
            fp16=not is_bfloat16_supported(),
            bf16=is_bfloat16_supported(),
            logging_steps=1,
            evaluation_strategy="steps",
            eval_steps=10,
            optim="adamw_8bit",
            weight_decay=0.01,
            lr_scheduler_type="linear",
            seed=3407,
            report_to="none",
        ),
    )

    # Start training
    trainer.train()

    # Save the final model
    trainer.save_model("./causal-inference-llama-3.1-final")

    # Optional: Merge adapter weights with the base model for deployment
    model.save_pretrained("lora_model") # Local saving
    tokenizer.save_pretrained("lora_model")

    # Test the model on a sample question
    test_question = "Is there a causal relationship between 'crude birth rate births per 1000 population' and 'all forms of tb number of new cases estimated'?"

    inputs = tokenizer(
        f"<|system|>\nYou are a causal inference expert that answers questions about causal relationships between variables based on data analysis. When asked about relationships, provide clear explanations about whether one variable causes another, is caused by another, or shares common causes with other variables. Interpret both the sign (positive/negative) and magnitude (strong/weak) of causal effects.\n<|user|>\n{test_question}\n<|assistant|>\n",
        return_tensors="pt"
    ).to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        temperature=0.7,
        do_sample=True,
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("\n\nSample model response:")
    print(response)

In [5]:
if __name__ == "__main__":
    main()

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

...

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 4,500 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
10,0.554300,0.463811
20,0.256500,0.258644
30,0.209900,0.211209
40,0.209000,0.190306
50,0.176000,0.170888
60,0.149100,0.163428


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient




Sample model response:
<|system|>
You are a causal inference expert that answers questions about causal relationships between variables based on data analysis. When asked about relationships, provide clear explanations about whether one variable causes another, is caused by another, or shares common causes with other variables. Interpret both the sign (positive/negative) and magnitude (strong/weak) of causal effects.
<|user|>
Is there a causal relationship between 'crude birth rate births per 1000 population' and 'all forms of tb number of new cases estimated'?
<|assistant|>
Based on the available data, there's no detected causal relationship from 'crude birth rate births per 1000 population' to 'all forms of tb number of new cases estimated'. However, the data shows that 'all forms of tb number of new cases estimated' slightly decreases 'crude birth rate births per 1000 population' with a weak effect (coefficient: -0.0012). This indicates that 'all forms of tb number of new cases es

In [3]:
from unsloth import FastLanguageModel
import torch
from transformers import TrainingArguments

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="/content/causal-inference-llama-3.1-final",
        max_seq_length=2048,
        dtype=None,  # Set to torch.bfloat16 or None based on your GPU capabilities
        load_in_4bit=True,  # Use 4-bit quantization to reduce memory requirements
    )
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

inputs = tokenizer(
[
    alpaca_prompt.format(
        "You are a causal inference expert that answers questions about causal relationships between variables based on data analysis. When asked about relationships, provide clear explanations about whether one variable causes another, is caused by another, or shares common causes with other variables. Interpret both the sign (positive/negative) and magnitude (strong/weak) of causal effects.", # instruction
        "Is there a causal relationship between 'crude birth rate births per 1000 population' and 'all forms of tb number of new cases estimated'?", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

Unsloth 2025.3.19 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are a causal inference expert that answers questions about causal relationships between variables based on data analysis. When asked about relationships, provide clear explanations about whether one variable causes another, is caused by another, or shares common causes with other variables. Interpret both the sign (positive/negative) and magnitude (strong/weak) of causal effects.

### Input:
Is there a causal relationship between 'crude birth rate births per 1000 population' and 'all forms of tb number of new cases estimated'?

### Response:
Based on the available data, there's no detected causal relationship between 'crude birth rate births per 1000 population' and 'all forms of tb number of new cases estimated'. However, the data shows that 'all forms of tb number of new cases estimated' ha